# Analysis

I analyze the `HOSP` datasets in this notebook.

In [1]:
import pandas as pd
from collections import Counter

from utils import get_differences

# Hospital

In [2]:
df_clean = pd.read_csv('../data/hospital/clean.csv')
df_dirty = pd.read_csv('../data/hospital/dirty.csv')

In [3]:
def find_replacement_rules(df, column):
    # Initialize a Counter to store rules and their counts
    replacement_rules = Counter()
    
    # Iterate over each row in the DataFrame
    for _, row in df.iterrows():
        clean = row[f'{column}_clean']
        dirty = row[f'{column}_dirty']
        
        # Check character-by-character for differences
        i = 0
        while i <= len(clean):
            if dirty[i] == 'x' and clean[i] != 'x':  # Found a character in clean replaced by 'x'
                replacement_rules[clean[i]] += 1
                i = len(clean)
            i += 1
    
    return replacement_rules

In [4]:
df_clean.columns

Index(['index', 'ProviderNumber', 'HospitalName', 'Address1', 'Address2',
       'Address3', 'City', 'State', 'ZipCode', 'CountyName', 'PhoneNumber',
       'HospitalType', 'HospitalOwner', 'EmergencyService', 'Condition',
       'MeasureCode', 'MeasureName', 'Score', 'Sample', 'StateAverage'],
      dtype='object')

- `index`: error-free.
- `ProviderNumber`: 28 errors, randomly replacing chars with `x` with frequencies, e.g.: `10115` becomes `101x5`.
- `HospitalName`: 24 errors, randomly replacing chars with `x` with frequencies:
    - `i`: 3
    - `o`: 1
    - `t`: 2
    - `k`: 1
    - `w`: 1
    - `f`: 1
    - `u`: 1
    - `a`: 2
    - `n`: 2
    - `c`: 2
    - `p`: 1
    - `y`: 1
    - `l`: 2
    - `s`: 1
    - `e`: 1
    - `r`: 1
    - `g`: 1
- `Address1`: 31 errors, randomly replacing chars with `x` with frequencies:
    - ` `: 11
    - `n`: 2
    - `t`: 3
    - `h`: 1
    - `i`: 2
    - `4`: 1
    - `r`: 1
    - `e`: 2
    - `o`: 2
    - `2`: 1
    - `8`: 1
    - `1`: 1
    - `d`: 1
    - `a`: 1
    - `b`: 1
- `Address2`: error-free.
- `Address3`: error-free.
- `City`: 33 errors, randomly replacing chars with `x` with frequencies:
    - 'e': 5,
    - 'a': 4,
    - 'i': 4,
    - 'l': 3,
    - 'n': 3,
    - 'g': 3,
    - 'o': 2,
    - 'y': 2,
    - 's': 2,
    - 'h': 1,
    - 'd': 1,
    - 'k': 1,
    - 't': 1,
    - 'r': 1
- `State`: 26 errors, randomly replacing chars with `x` with frequencies
    - 'a': 16,
    - 'l': 10
- `ZipCode`: 30 errors, randomly replacing chars with `x` with frequencies
    - {'3': 9, '5': 8, '6': 4, '1': 3, '0': 3, '7': 1, '4': 1, '2': 1}
- `CountyName`: 39 errors, randomly replacing chars with `x` with frequencies
    - 'e': 9,
     't': 4,
     'r': 3,
     'o': 3,
     'd': 3,
     'u': 3,
     'l': 3,
     'a': 2,
     'c': 2,
     'f': 2,
     'm': 2,
     'j': 1,
     'v': 1,
     'n': 1
- `PhoneNumber`: 34 errors, randomly replacing chars with `x` with frequencies
    - '2': 7,
         '1': 5,
         '4': 5,
         '3': 4,
         '5': 3,
         '0': 3,
         '9': 2,
         '6': 2,
         '8': 2,
         '7': 1
- `HospitalType`: 32 errors, randomly replacing chars with `x` with frequencies
    - 't': 8,
         ' ': 7,
         'p': 5,
         'e': 2,
         'l': 2,
         'r': 2,
         'c': 2,
         'u': 1,
         'o': 1,
         'a': 1,
         's': 1
- `EmergencyService`: 27 errors, randomly replacing chars with `x` with frequencies
- {'e': 11, 's': 9, 'y': 5, 'n': 2}

**... and so on**. Rules can be read programmatically, the error type appears to be always the same.

In [5]:
df_diff = get_differences(df_clean, df_dirty, 'ProviderNumber')
df_diff_nona = df_diff[~df_diff['ProviderNumber_dirty'].isna()]
len(df_diff[df_diff_nona['ProviderNumber_dirty'].str.contains('x')])

28

In [6]:
df_diff = get_differences(df_clean, df_dirty, 'HospitalName')
df_diff_nona = df_diff[~df_diff['HospitalName_dirty'].isna()]
len(df_diff)

24

In [7]:
df_diff = get_differences(df_clean, df_dirty, 'Address1')
df_diff_nona = df_diff[~df_diff['Address1_dirty'].isna()]
len(df_diff)

31

In [8]:
col = 'City'
df_diff = get_differences(df_clean, df_dirty, col)
df_diff_nona = df_diff[~df_diff[f'{col}_dirty'].isna()]
find_replacement_rules(df_diff, col)

Counter({'e': 5,
         'a': 4,
         'i': 4,
         'l': 3,
         'n': 3,
         'g': 3,
         'o': 2,
         'y': 2,
         's': 2,
         'h': 1,
         'd': 1,
         'k': 1,
         't': 1,
         'r': 1})

In [9]:
col = 'ZipCode'
df_clean[col] = df_clean[col].astype('str')
df_diff = get_differences(df_clean, df_dirty, col)
find_replacement_rules(df_diff, col)

Counter({'3': 9, '5': 8, '6': 4, '1': 3, '0': 3, '7': 1, '4': 1, '2': 1})

In [10]:
col = 'CountyName'
df_diff = get_differences(df_clean, df_dirty, col)
find_replacement_rules(df_diff, col)
len(df_diff)

39

In [11]:
col = 'PhoneNumber'
df_clean[col] = df_clean[col].astype('str')
df_diff = get_differences(df_clean, df_dirty, col)
find_replacement_rules(df_diff, col)
len(df_diff)

34

In [12]:
col = 'HospitalType'
df_diff = get_differences(df_clean, df_dirty, col)
find_replacement_rules(df_diff, col)
len(df_diff)

32

In [13]:
col = 'EmergencyService'
df_diff = get_differences(df_clean, df_dirty, col)
find_replacement_rules(df_diff, col)
len(df_diff)

27

In [14]:
col = 'StateAverage'
df_diff = get_differences(df_clean, df_dirty, col)


In [15]:
df_diff

,StateAverage_clean,StateAverage_dirty
56,al_pn-3b,al_pn-xb
69,al_ami-1,al_amx-1
193,al_scip-vte-2,al_scix-vte-2
213,al_scip-inf-2,al_sxip-inf-2
244,al_ami-1,ax_ami-1
248,al_ami-5,xl_xmi-5
269,al_ami-1,al_axi-1
334,al_pn-2,al_pnx2
347,al_scip-vte-2,al_scipxvtex2
370,al_pn-7,alxpn-7
